<a href="https://colab.research.google.com/github/aidamash/Pytorch-UNet/blob/master/hummingbird.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import gc
import os

import time
import tqdm
import collections
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm.auto import tqdm as tq
from sklearn.model_selection import train_test_split

!pip install rasterio
import rasterio as rst
import rasterio.features
import rasterio.warp
from rasterio.plot import show
from rasterio.windows import Window
from rasterio.plot import reshape_as_raster, reshape_as_image

     |████████████████████████████████| 19.1MB 163kB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [8]:
#read and visualise images
src = rasterio.open('drive/MyDrive/hummingbird/all/apollo1.tif')
src.name

#find the shape
array = src.read()
array.shape, type(array)


((3, 2769, 3659), numpy.ndarray)

In [10]:
img_path = 'drive/MyDrive/hummingbird/all/'
img_list = []
for filename in os.listdir(img_path):
        # find images
        img_list.append(img_path + filename)

img_list

['drive/MyDrive/hummingbird/all/apollo3.tif',
 'drive/MyDrive/hummingbird/all/apollo2.tif',
 'drive/MyDrive/hummingbird/all/apollo1.tif',
 'drive/MyDrive/hummingbird/all/littlegems2.tif',
 'drive/MyDrive/hummingbird/all/littlegems3.tif']

In [ ]:
width=218
height=218
rows = 2
cols = 3
axes=[]
fig=plt.figure()

for i, img in enumerate(img_list):
    print(img)
    src = rst.open(img)
    b = reshape_as_image(src.read())
    #axes.append(fig.add_subplot(rows, cols, a+1) )
    plt.subplot(3, 2, i+1)
    plt.imshow(b)

fig.tight_layout()    
plt.show()




In [11]:
labels = ['apollo', 'littlegems']
# dict that maps labels to integers, and the reverse
labels_map = {labels[i]:i for i in range(len(labels))}
inv_labels_map = {i:labels[i] for i in range(len(labels))}
labels_map,inv_labels_map

({'apollo': 0, 'littlegems': 1}, {0: 'apollo', 1: 'littlegems'})

# New Section

Load keras and tensorflow related libs

In [12]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

import sys
from numpy import load
from matplotlib import pyplot
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction import image
from keras import backend
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import Flatten
from keras.optimizers import SGD
from tensorflow.keras import initializers

In [13]:
# define cnn model
def define_model(in_shape=(128, 128, 3), out_shape=1):
	model = Sequential()
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='random_normal', padding='same', input_shape=in_shape))
	model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='random_normal', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='random_normal', padding='same'))
	model.add(Conv2D(64, (3, 3), activation='relu', kernel_initializer='random_normal', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='random_normal', padding='same'))
	model.add(Conv2D(128, (3, 3), activation='relu', kernel_initializer='random_normal', padding='same'))
	model.add(MaxPooling2D((2, 2)))
	model.add(Flatten())
	model.add(Dense(128, activation='relu', kernel_initializer=initializers.RandomNormal(stddev=0.01),bias_initializer='zeros' ))

	model.add(Dense(out_shape, activation='sigmoid'))
	# compile model
	opt = keras.optimizers.SGD(lr=0.001, momentum=0.9)
	model.compile(optimizer=opt, loss='binary_crossentropy')
	return model

In [25]:
from rasterio.windows import Window
import numpy as np
width = 128
height = 128
def load_dataset(path):
    photos = list()
    labels = []
    mapping=dict()
    # enumerate files in the directory
    for filename in os.listdir(path):
      # load image
      print(path + filename)
      src = rst.open(path + filename)
      
      # convert to numpy array
      photo = src.read(window=Window(0, 0, width, height))
      photo = reshape_as_image(photo)
      profile = src.profile
      profile['width'] = width
      profile['height'] = height
      print(photo.shape)

      # get labels
      if 'apollo' in filename:
          labels.append(0)
          mapping[filename] = 0
      else:
          labels.append(1)
          mapping[filename] = 1
      #store
      photos.append(photo)
      X = np.asarray(photos, dtype='uint8')
      y = np.asarray(labels, dtype='uint8')
    return X, y


In [26]:
X, y = load_dataset("drive/MyDrive/hummingbird/all/")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
# create data generator
datagen = ImageDataGenerator(rescale=1.0/255.0,
                             horizontal_flip=True,
                             height_shift_range=0.5,
                             rotation_range=90)
# prepare iterators
train_it = datagen.flow(X_train, y_train, batch_size=1)
test_it = datagen.flow(X_test, y_test, batch_size=1)
# # define model
model = define_model()


drive/MyDrive/hummingbird/all/apollo3.tif
(128, 128, 3)
drive/MyDrive/hummingbird/all/apollo2.tif
(128, 128, 3)
drive/MyDrive/hummingbird/all/apollo1.tif
(128, 128, 3)
drive/MyDrive/hummingbird/all/littlegems2.tif
(128, 128, 3)
drive/MyDrive/hummingbird/all/littlegems3.tif
(128, 128, 3)
drive/MyDrive/hummingbird/all/littlegems1.tif
(128, 128, 3)


In [32]:
# epochs = 10

# checkpoint_filepath = 'model/'
# model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
#     filepath=checkpoint_filepath,
#     monitor='val_accuracy',
#     mode='max')

# model.compile(
#     optimizer=keras.optimizers.SGD(),
#     loss="binary_crossentropy",
#     metrics=["accuracy"],
# )
# model.fit(
#     train_it, epochs=epochs, callbacks=model_checkpoint_callback, validation_data=test_it,
# )

In [33]:
def load_dataset(path):
    photos = list()
    labels = []
    mapping = dict()
    # enumerate files in the directory
    for filename in os.listdir(path):
      # load image
      print(path + filename)
      src = rst.open(path + filename)

      # convert to numpy array
      photo = src.read()
      photo = reshape_as_image(photo)
      print(photo.shape)
      patches = image.extract_patches_2d(photo, (128, 128))

      # get tags
      if 'apollo' in filename:
          labels.append(0)
          mapping[filename] = 0
      else:
          labels.append(1)
          mapping[filename] = 1
      

      photos.append(photo)
      
      #targets.append(target)
      X = np.asarray(photos, dtype='uint8')
      y = np.asarray(labels, dtype='uint8')
    return X, y


In [1]:
X, y = load_dataset("drive/MyDrive/hummingbird/all/")
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33)
# create data generator
# datagen = ImageDataGenerator(rescale=1.0/255.0,
#                              horizontal_flip=True,
#                              height_shift_range=0.5,
#                              rotation_range=90)
# # prepare iterators
# train_it = datagen.flow(X_train, y_train, batch_size=1)
# test_it = datagen.flow(X_test, y_test, batch_size=1)
# # # define model
# model = define_model()



# src = rst.open('dataset/all/apollo1.tif')
# from sklearn.feature_extraction import image    
# import patchify
# # convert to numpy array
# photo = src.read()
# photo = reshape_as_image(photo)
# photo.shape
# patches = image.extract_patches_2d(photo, (3, 3))
# print(photo.shape), print('Patches shape: {}'.format(patches.shape))
# print(patches[800])

NameError: ignored